In [ ]:

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini",
                   api_key=input("Enter your OpenAI API key: "))

In [ ]:
model.invoke("What's on the front page of hacker news?").content

In [36]:
from langchain_core.tools import tool
import requests

@tool
def read_webpage(url: str) -> str:
    """
    Fetch a url's content and return it as a string
    """
    print("Fetching content from", url)
    response = requests.get(url)
    
    return response.text


tools = [read_webpage]

In [37]:
tool_model = model.bind_tools(tools)

In [ ]:
tool_model.invoke("what's on the front page of hacker news?")

In [40]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

system_prompt = """
    You must provide your answers in a format readable in a terminal. Do not provide any text other than the answer.
    Your responses must not exceed 120 characters in width but can be multiple lines.
"""


def call_agent(state: MessagesState):
    response = tool_model.invoke([
        ("system", system_prompt),
        *state["messages"]
    ])

    return {
        "messages": [response]
    }


def after_prompt_edge(state: MessagesState):
    if state["messages"][-1].tool_calls:
        return "tools"
    return END


tool_node = ToolNode(tools)

workflow = StateGraph(MessagesState)

workflow.add_node("prompt", call_agent)

workflow.add_node("tools", tool_node)

workflow.add_edge(START, "prompt")
workflow.add_edge("tools", "prompt")
workflow.add_conditional_edges("prompt", after_prompt_edge)

app = workflow.compile()

In [ ]:
messages = [
    HumanMessage(content="Return the top 10 stories on hacker news, their titles, URLs, and comment URLs.")
]

final_state = app.invoke(
    {"messages": messages},
)

messages.append(final_state['messages'][-1])

print(final_state['messages'][-1].content)

In [ ]:

messages.append(HumanMessage(content=input("What is your follow-up question?")))

final_state = app.invoke(
    {"messages": messages},
)

messages.append(final_state['messages'][-1])

print(final_state['messages'][-1].content)

In [ ]:
state = app.invoke({"messages": [
	HumanMessage(content="Using the Pokemon API, return the name of the 5 latest pokemon, their types, and their description."),
]})

print(state['messages'][-1].content)